### Improved CNN for FT of MNIST data. Also padded size.

### v3 introduces 2 channels of intensity and phase to see if accuracy improves

### v4 introduces hyperas to do hyper-param optimization

In [1]:
#Choose GPUs and CPUs

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist

#If want to choose GPU to use
#This will make it so that only the 0th GPU is visible to TensorFlow.
#In your case, you can choose any in the range [0, 3].
#If you wanted, for example, the 0th and 2nd GPUs to both be visible, replace "0" with "0,2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

GPU=True
CPU=False
num_cores = 8

if GPU:
    num_GPU = 2
    num_CPU = 2
if CPU:
    num_CPU = num_cores
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU},\
#        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))
                       )
config.gpu_options.allow_growth = True

session = tf.Session(config=config)
K.set_session(session)


Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import numpy as np

from keras.layers import Conv2D, MaxPool2D
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import to_categorical, multi_gpu_model

from tqdm import tqdm_notebook
from scipy.fftpack import fftn,fftshift


from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice,uniform


# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

# Load MNIST data from Keras

In [3]:
def data():
    # input image dimensions
    img_rows, img_cols = 28, 28
    num_classes=10

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    train_dataset = x_train.astype('float32')
    test_dataset = x_test.astype('float32')
    train_dataset /= 255
    test_dataset /= 255
    print('train_dataset shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    train_labels = keras.utils.to_categorical(y_train, num_classes)
    test_labels = keras.utils.to_categorical(y_test, num_classes)


    # Pad numpy array
    # Want 36x36 which is how big the multiMNIST will be
    train_dataset=np.pad(train_dataset, ((0,0),(4,4),(4,4),(0,0)), 'constant', constant_values=(0,0))
    print (train_dataset.shape)

    test_dataset=np.pad(test_dataset, ((0,0),(4,4),(4,4),(0,0)), 'constant', constant_values=(0,0))
    print (test_dataset.shape)

    # Take FT
    dims=train_dataset.shape
    ft_train=np.zeros((dims[0], 36, 36, 2), float)
    for i in range(dims[0]):
        ft_train[i,:,:,0]=np.abs(fftshift(fftn(train_dataset[i,:,:]))).squeeze()
        ft_train[i,:,:,1]=np.angle(fftshift(fftn(train_dataset[i,:,:]))).squeeze()
    #    ft_train[i,:,:,1]=unwrap_phase(ft_train[i,:,:,1])

    dims=test_dataset.shape
    ft_test=np.zeros((dims[0], 36, 36, 2), float)
    for i in tqdm_notebook(range(dims[0])):
        ft_test[i,:,:,0]=np.abs(fftshift(fftn(test_dataset[i,:,:]))).squeeze()
        ft_test[i,:,:,1]=np.angle(fftshift(fftn(test_dataset[i,:,:]))).squeeze()
    #    ft_test[i,:,:,1]=unwrap_phase(ft_test[i,:,:,1])


    dims=ft_train.shape
    print (dims)
    trainee=ft_train.reshape(dims[0],dims[1],dims[2],2)
    dims=test_dataset.shape
    testee=ft_test.reshape(dims[0],dims[1],dims[2],2)
    print (trainee.shape)
    return trainee, train_labels, testee, test_labels

# O.K now try CNN

In [4]:
"""
class ModelMGPU(Model):
    def __init__(self, ser_model, gpus):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)
"""

"\nclass ModelMGPU(Model):\n    def __init__(self, ser_model, gpus):\n        pmodel = multi_gpu_model(ser_model, gpus)\n        self.__dict__.update(pmodel.__dict__)\n        self._smodel = ser_model\n\n    def __getattribute__(self, attrname):\n        '''Override load and save methods to be used from the serial-model. The\n        serial-model holds references to the weights in the multi-gpu model.\n        '''\n        # return Model.__getattribute__(self, attrname)\n        if 'load' in attrname or 'save' in attrname:\n            return getattr(self._smodel, attrname)\n\n        return super(ModelMGPU, self).__getattribute__(attrname)\n"

In [5]:
def create_model(trainee, train_labels, testee, test_labels):

    w_size={{choice([2,3,4])}}

    model=Sequential()

    model.add(Conv2D(32,(w_size,w_size), activation='relu', input_shape=(36,36,2),
                     data_format='channels_last', padding='same'))
    model.add(Conv2D(32,(w_size,w_size), activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(64,(w_size,w_size), activation='relu', padding='same'))
    model.add(Conv2D(64,(w_size,w_size), activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(128,(w_size,w_size), activation='relu', padding='same'))
    model.add(Conv2D(128,(w_size,w_size), activation='relu', padding='same'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(600, activation='relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(10, activation='sigmoid')) 

    #parallel_model = ModelMGPU(model, gpus=num_GPU)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history=model.fit(trainee, train_labels, batch_size=256, verbose=0, epochs=10, 
                  validation_split = 0.1)#, callbacks=[checkpoints])
    
    score=model.evaluate(testee, test_labels)
    
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(history.history['val_acc'])
    print(validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}
    
best_run, best_model = optim.minimize(model=create_model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=Trials(),
                          notebook_name='v4_hyperas_optimized')

>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    import os
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.layers import Conv2D, MaxPool2D
except:
    pass

try:
    from keras.models import Sequential, Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.utils import to_categorical, multi_gpu_model
except:
    pass

try:
    from tqdm import tqdm_notebook
except:
    pass

try:
    from scipy.fftpack import fftn, fftshift
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
e

/home/mcherukara/miniconda3/envs/gpu/lib/python3.6/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x



(60000, 36, 36, 2)
(60000, 36, 36, 2)
10000/10000 [==============================] - 1s 67us/step
0.9823333328564962
10000/10000 [==============================] - 0s 47us/step
0.9814999995231628
10000/10000 [==============================] - 1s 52us/step
0.9801666668256124
10000/10000 [==============================] - 0s 47us/step
0.9813333328564962
10000/10000 [==============================] - 0s 46us/step
0.9829999995231629
10000/10000 [==============================] - 1s 67us/step
0.9834999998410543
10000/10000 [==============================] - 0s 47us/step
0.9835000001589457
10000/10000 [==============================] - 1s 68us/step
0.9835000001589457
10000/10000 [==============================] - 0s 50us/step
0.9833333328564962
10000/10000 [==============================] - 1s 54us/step
0.9843333328564962


In [7]:
trainee, train_labels, testee, test_labels=data()

train_dataset shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000, 36, 36, 1)
(10000, 36, 36, 1)



(60000, 36, 36, 2)
(60000, 36, 36, 2)


In [8]:
#from keras.models import Model, load_model
#model=load_model('ft_models/weights.06.hdf5') #Numbering starts from 0
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score=best_model.evaluate(testee, test_labels, verbose=1)
print (score)

10000/10000 [==============================] - 1s 55us/step
[0.07110347990648005, 0.9816]


In [9]:
print (best_run)
print (best_model.summary())

{'Dropout': 0.22780041552649732, 'w_size': 1}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 36, 36, 32)        608       
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 18, 18, 64)        18496     
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_59 (Conv2D)           (

In [ ]:
preds=(model.predict(testee))

In [ ]:
i=100
f,axarr=plt.subplots(1,2,figsize=(8, 6))
axarr[0].imshow(test_dataset[i].squeeze(),cmap='viridis')
img=axarr[1].imshow(testee[i,:,:,1].squeeze(), cmap='viridis')
plt.colorbar(img, fraction=0.046, pad=0.04)
print (np.argmax(preds[i]))